# **Ансамблирование: бустинг**

**Бустинг (boosting)** — это алгоритм построения ансамбля, основанный на *последовательном* построении *слабых* моделей, причём каждая новая модель пытается *уменьшить ошибку предыдущей*. После того как все модели обучены, они объединяются в композицию.

*Примечание. Под слабыми моделями мы подразумеваем модели, точность которых немногим выше, чем случайное угадывание. Как правило, это короткие деревья решений, они обладают слабой предсказательной способностью.*
***
Обратите внимание, что в бустинге базовые модели обучаются последовательно, а не параллельно, как в предыдущих методах, исправляя ошибки своего «предшественника» и повышая качество всего ансамбля. 

Бустинг основан на вопросе, поднятом исследователями М. Кернсом и Л. Вэлиантом: «Может ли набор слабых обучающих алгоритмов создать сильный обучающий алгоритм?»

В отличие от бэггинга, бустинг обучается на одном и том же наборе данных, без генерации дополнительных выборок. Однако в процессе обучения меняются так называемые веса наблюдений. Если слабая модель допустила ошибку на каких-то примерах, то значимость (вес) этих примеров увеличивается и на них концентрируется следующая за ней модель.

Представить алгоритм бустинга можно следующей схемой:

![](https://lms.skillfactory.ru/assets/courseware/v1/f57e9e2fc278d940a8a1074e62b904f3/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-ml3-7_1.png)

На схеме  представлено 11 разных наблюдений трёх различных классов (красные, зелёные и синие шарики). После того как модель делает предсказания, мы смотрим, на каких объектах мы угадали класс верно, а на каких ошиблись. Для тех объектов, на которых мы допустили ошибку, мы задаём больший вес. Вес наблюдения обозначается интенсивностью цвета. Чем больше вес наблюдения, тем ярче его цвет.
***

Так же как и бэггинг, бустинг предназначен для **обучения моделей одного типа**. То есть нельзя последовательно обучить 50 логистических регрессий, а затем 50 деревьев решения.

Каждая модель создаётся для того, чтобы найти ошибки предыдущей. Сами по себе они решают задачу плохо, но стоит объединить их усилия, и мы получим **супермодель**.

Очень наглядно будет выглядеть модель бустинга для логистической регрессии при обучении на двух признаках:

![](https://lms.skillfactory.ru/assets/courseware/v1/46bb0f960392c6a39fcf52d6da588054/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-ml3-7_2.png)

На рисунке изображено расположение двух линейно неразделимых классов. Нельзя провести одну такую плоскость, которая идеально решает задачу классификации. Раз нельзя одну, давайте проведём две.

Возьмём точки, для которых первая логистическая регрессия совершила ошибку. Увеличим вес этих точек (их значимость). Далее построим такую разделяющую плоскость, которая в первую очередь обращает внимание на наблюдения, имеющие наибольший вес.

Наконец, объединим две модели в одну — получим модель-композицию, которая идеально решает задачу классификации!

*Примечание. Когда все модели из ансамбля обучены и составлена композиция из них, для того, чтобы совершить предсказание на новом объекте, необходимо «прогнать» характеристики объекта через все модели в той же последовательности, в которой они обучались, и объединить их результат.*
***

Если бэггинг создавался с целью уменьшить разброс модели, то цель бустинга — **уменьшить смещение модели**.

Каждый следующий базовый алгоритм в бустинге обучается так, чтобы уменьшить общую ошибку всех своих предшественников. Как следствие, итоговая композиция будет иметь меньшее смещение, чем каждый отдельный базовый алгоритм (хотя уменьшение разброса также может происходить).

В предельном случае модель может обучиться так, что не будет допускать ошибок вовсе. Однако мы знаем, что это не всегда хорошо, ведь в таком случае модель может полностью подстроиться под обучающий набор данных и переобучиться.

Поскольку основная цель бустинга — уменьшение смещения, в качестве базовых моделей часто выбирают алгоритмы с высоким смещением и небольшим разбросом, например короткие деревья решений. У каждого из таких деревьев слабая предсказательная способность, но если их объединить, мы получим очень мощную модель. 

В этом юните мы постараемся затронуть основные шаги эволюции бустинга от первой успешной модели до современных модификаций. Начнём рассмотрение с самой первой модели бустинга — **адаптивного бустинга**.

***
## **АДАПТИВНЫЙ БУСТИНГ**

Первая реализация бустинга называлась **AdaBoost**. Это модель, которая подразумевает воплощение той самой идеи взвешивания объектов, которую мы рассмотрели выше. Алгоритм предполагает постоянную модификацию объектов выборки путём их взвешивания, причём веса обновляются специальным образом: каждая новая модель из ансамбля обучается на взвешенных данных и обращает большее внимание на ошибки своих предшественников.

Так как алгоритм является несовершенным и в дальнейшем получил свое развитие, мы не будем подробно останавливаться на его работе. Однако приведем краткое описание работы алгоритма на примере **задачи бинарной классификации**. 

Пусть у нас есть набор данных ***X***, в котором ***N*** объектов размерности ***m*** (вектора в признаковом пространстве размера ***M***) и метки класса ***y ∈ -1, 1***, где -1 и 1 — метки отрицательного и положительного класса соответственно.

Будем строить ансамбль из ***K*** абстрактных базовых моделей — классификаторов. Обозначим их как ***a(x)*** (это могут быть логистические регрессии/деревья решений или что-то ещё).  

1. Перед обучением базовой модели мы инициализируем веса объектов обучающей выборки следующим образом:

![](data/f1.PNG)

2. Для всех ***i*** от 1 до ***K***:

   1. Обучить базовую модель ***ai(x)*** с учётом весов объектов ***wj***.

   2. Вычислить ошибку классификатора ***ai(x)***, обозначим её за ***εj***:

![](https://lms.skillfactory.ru/assets/courseware/v1/b2cacd56325fd9e043f8cfc25a10fd94/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst-3-ml-8-59.png)

Примечание. Здесь выражение ![](data/f2.PNG) — это знакомая нам по модулю классификации индикаторная функция. Она равна 1, если ответ ***yj*** не совпал с предсказанием базовой модели ***ai(x)***, и 0 — в противном случае.

3. Тогда вес предсказаний данного классификатора (мера «вклада» предсказаний i-ой модели в общий ансамбль) вычисляется по формуле: 

![](data/f3.PNG)

Формула веса становится нулевой только при **случайном угадывании**, то есть когда классификатор ошибается в половине меток — работает ровно так же, как и подбрасывание монетки. Однако остальные будут вносить вес в итоговую модель с положительным или отрицательным знаком (в этом легко убедиться, подставив вероятность ошибки, отличную от 0.5). Тем самым мы исключаем возможность вклада случайных классификаторов в результирующую модель.

4. Обновляем веса объектов в выборке. Для тех объектов, на которых мы допустили ошибку, вес увеличивается; для тех объектов, для которых наш ансамбль предсказал верный ответ — не изменяется. Формула обновления весов:

![](https://lms.skillfactory.ru/assets/courseware/v1/8fd37ebf163d92d4f96f358e6b37a414/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst-3-ml-8-60.png)

Видно, что если классификация была произведена верно для объекта ***xj***, то степень экспоненты будет равна 0, а значит множитель ![](data/f4.PNG) и вес  не изменится. 

Для того, чтобы привести все веса объектов к единому масштабу от 0 до 1, производится их нормировка путем деления каждого веса ***wj*** на сумму всех весов:

![](data/f5.PNG)

5. После обучения каждой базовой модели предсказания ансамбля строятся как сумма из предсказаний базовых моделей, взятых с весом ![](data/f6.PNG)

Примечание. Функция ***sign*** — функция взятия знака, принимает значение -1, если аргумент функции отрицательный, 0 — если аргумент функции нулевой, и 1 — если аргумент функции положительный.

![](data/f7.PNG)
***

**В чём плюсы такого алгоритма?**

✔️ **Он прост**. Обратите внимание: все математические операции — школьный курс математики, о высшей математике даже не идёт речи. Операции просты в реализации и не требуют вычисления производных, умножений матриц и прочих сложных математических конструкций.

✔️ Накладные **расходы бустинга минимальны**. Время построения определяется временем построения базовых моделей.

✔️ Показывает **хорошую обобщающую способность**.

✔️ Имеет возможность **идентификации шумовых объектов**.

**Но в чём минусы?**

⛔️ Жадное добавление алгоритмов приводит к **неоптимальности композиции**.

⛔️ Склонен к **переобучению** при наличии шума в данных.

⛔️ Алгоритм является эвристикой, и «взвешивание» объектов, на котором он основан, **не подкреплено математическим обоснованием**.

***
## **АДАПТИВНЫЙ БУСТИНГ В SKLEARN**

В **sklearn** адаптивный бустинг над решающими деревьями реализован в модуле **sklearn.ensemble** в виде классов [**AdaBoostRegressor**](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html) и [**AdaBoostClassifier**](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier) для задач регрессии и классификации соответственно. Давайте проведём обучение на тех же данных, что и в предыдущих моделях ансамблирования — на данных о диабете. 

Прежде чем перейти к практической части, предлагаем ознакомиться с **параметрами AdaBoost**:

* **base_estimator** — параметр отвечает за природу базовых моделей, по умолчанию это DecisionTreeRegressor c максимальной глубиной (max_depth) 3.
* **n_estimators** — максимальное количество базовых моделей, по умолчанию равно 50. В случае идеального обучения алгоритм завершается ранее, чем данное значение.
* **learning_rate** — темп обучения, параметр, добавляющий дополнительный множитель весу базовой модели, по умолчанию он равен 1.
* **loss**{'linear', 'square', 'exponential'} — функция ошибки для обновления весов (в теоретической части мы рассматривали экспоненциальную форму обновления весов — 'exponential')
* **random_state** — параметр, фиксирующий случайные процессы в модели.

Для сравнимости результатов со случайным лесом возьмём количество базовых моделей, равное 10. Как говорилось ранее, глубина деревьев должна быть меньше, чем у случайного леса. По умолчанию она равна 3.

In [21]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.datasets import load_diabetes
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
data = load_diabetes(as_frame=True)
X = data['frame']
y = data['target']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [4]:
from sklearn.ensemble import AdaBoostRegressor
dt = DecisionTreeRegressor(
    max_depth=3, 
    random_state=42
)

ada = AdaBoostRegressor(
    base_estimator=dt,
    random_state=42, 
    n_estimators=10
)

ada.fit(X_train, y_train)
 
 
ada_pred  = ada.predict(X_test)
print(f'Качество предсказания по MSE для AdaBoost {round(mean_squared_error(y_test, ada_pred),2)}')

Качество предсказания по MSE для AdaBoost 40.5


In [6]:
# Измените количество базовых моделей до 50 и 100.
# В качестве ответа приведите полученные результаты,
# округленные до второго знака после запятой. Остальные параметры оставьте неизменными.

ada = AdaBoostRegressor(
    base_estimator=dt,
    random_state=42, 
    n_estimators=100
)

ada.fit(X_train, y_train)
 
 
ada_pred  = ada.predict(X_test)
print(f'Качество предсказания по MSE для AdaBoost {round(mean_squared_error(y_test, ada_pred),2)}')

Качество предсказания по MSE для AdaBoost 10.15


Основное отличие **градиентного бустинга** от **адаптивного** заключается в том, что градиентный бустинг строит композицию из своих базовых моделей, подбирая их оптимальным образом на основе **принципа градиентных методов оптимизации**.

Такая модификация позволяет значительно ускорить процесс последовательного построения ансамбля в сравнении с реализацией AdaBoost и добиться лучшего качества за счет меньшего количества моделей в ансамбле. 

***
## **ГРАДИЕНТНЫЙ БУСТИНГ**

* **Градиентный бустинг (Gradient Boosting, GB)** — это наиболее обобщённая версия бустинга, закреплённая математическим обоснованием. Впервые алгоритм был опубликован профессором статистики Стэнфордского университета Джеромом Фридманом. Алгоритм оказался очень эффективным и в дальнейшем был множество раз модифицирован — до **Extreme Gradient Boosting (XgBoost)** и других модификаций, таких как **CatBoost** от Яндекса и **LightGMB** от Microsoft.

Сейчас градиентный бустинг и его модификации применяются практически везде. Любой запрос на Яндексе, выбор отеля на Booking или сериала на Netflix — всё это работает на градиентном бустинге. 

→ В этом модуле мы кратко рассмотрим принцип работы, а в модулях по математике разберём математическую формализацию алгоритма. 

В **GB** принцип классического бустинга сохраняется: каждый последующий алгоритм улучшает предыдущий, но, в отличие эвристического «взвешивания» наблюдений, градиентный бустинг использует **информацию о функции потерь** для построения нового алгоритма. 

Допустим, у нас есть некоторая функция потерь ***L(y,y^)***. Она зависит от двух аргументов: ***y*** — истинный ответ, ***y^ = a(x)*** — прогноз модели . Причём неважно, какая это функция потерь (на самом деле ограничение есть — это дифференцируемость функции, то есть существование производной).

Для задачи регрессии это может быть, например, **MSE**:

![](data/f8.PNG)

Для задачи классификации это может быть, например, **logloss**:

![](data/f9.PNG)

Примечание. Для logloss ***y^*** в данном контексте означает не предсказанный класс для ***i***-ого наблюдения, а вероятность принадлежности к классу 1 ***i***-ого наблюдения, то есть на самом деле ***yi = P***i. Чистая формальность для идентичности записей для регрессии и классификации.

Пусть мы построили какую-то модель, которая решает задачу классификации или регрессии. Обозначим её ***a1(x)*** — это какая-то функция, которая принимает на вход объекты ***x*** и выдаёт для них предсказания. Качество такой модели, скорее всего, будет не очень хорошим, ведь мы обучаем слабые модели.

Однако мы знаем, на каких объектах модель давала точные предсказания, а на каких — ошибалась. Нужно попробовать использовать эту информацию и обучить ещё одну модель ***a2(x)***.

Допустим, мы знаем, что предсказание первой модели на каком-то объекте ***xi*** на 10 больше, чем в реальности. То есть ***a1(xi) = yi + 10***. То есть нам необходимо обучить следующую модель ***a2(x)*** таким образом, чтобы она предсказала для ***xi*** ответ -10, то есть ***a2(x) = -10***. Тогда предсказание идеально совпадает с действительностью:

![](data/f10.PNG)

Другими словами, следующая модель должна научиться предсказывать ошибки предыдущей.

В реальности вторая модель тоже не сможет обучиться идеально, поэтому обучим третью модель ***a3(x)***, которая будет «компенсировать» неточности первых двух. Будем продолжать так, пока не построим композицию из ***K*** алгоритмов.

Финальная модель (обозначим её за ***f(x)***), ищется в виде композиции из ***K*** базовых алгоритмов определённого семейства (например, линейных моделей/деревьев):

![](data/f11.PNG)

***
![](data/f12.PNG)
![](https://lms.skillfactory.ru/assets/courseware/v1/569173e552ccd595bfd01348fbecfc4d/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-ml3-7_3.png)

                                                        * * *

В качестве базовой модели можно использовать всё что угодно, но общепринятым является использование деревьев решений. Практика показывает, что это наилучший выбор, так как деревья решений очень просты в построении и из всех слабых моделей обладают наилучшей способностью описывать сложные зависимости.

Бустинг, использующий в качестве базовой модели дерево решений, называется **градиентным бустингом над деревьями решений (Gradient Boosting on Decision Trees, GBDT)**. 

Схематично работу алгоритма GBDT можно представить следующим образом:

![](https://lms.skillfactory.ru/assets/courseware/v1/d041c24e7a23daa059f376cd82343aee/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-ml3-7_5.png)

Основным преимуществом такой схемы градиентного бустинга является эффективность в поиске нелинейных зависимостей в сравнении с любыми моделями, основанными на решающих деревьях. Это преимущество стало причиной доминирования GBDT на огромном спектре соревнований — от кредитного скоринга до рекомендательных систем.

В заключение введения в градиентный бустинг приведём некоторые рекомендации по выбору внешних параметров алгоритма:

* Количество деревьев (**n_estimators**). Чем больше деревьев вы берёте, тем меньше ошибка на обучающем наборе данных, вплоть до 0, но, как вы понимаете, тем выше шанс переобучиться. Лучше начинать с небольшого количества моделей (50-100), а затем следить за ошибкой на тестовой выборке.
* Темп обучения  (**learning_rate**). Чем выше темп обучения, тем больше вклад каждого следующего дерева будет в модель и тем быстрее вы сойдётесь к минимуму функции потерь и сведёте ошибку к 0. Однако снова высок риск переобучения. Рекомендуемые значения — от 0.01 до 1.
* Максимальная глубина деревьев (**max_depth**). Градиентный бустинг лучше всего работает со слабыми моделями — это короткие деревья решений с глубиной от 1 до 8.
 
Все параметры влияют на обучение комплексно, поэтому их следует подбирать одновременно. О том, какие инструменты для этого существуют, мы поговорим в отдельном модуле.
***

Чтобы понять, как на градиентный бустинг влияют параметры темпа обучения, максимальной глубины деревьев и количества деревьев, предлагаем вам поиграть с настройками градиентного бустинга интерактивной демонстрации [**Brilliantly wrong**](http://arogozhnikov.github.io/2016/07/05/gradient_boosting_playground.html).

Вы можете регулировать следующие параметры:

* tree depth — максимальная глубина деревьев;
* learning rate — темп обучения;
* subsample — процент выборки, отведённый на обучение;
* trees — количество деревьев.

На демонстрации вы сможете увидеть, как меняется функция ошибки на тренировочной (train loss) и тестовой (test loss) выборках при изменении параметров, а также как меняется вид разделяющей поверхности.

Пример:

![](https://lms.skillfactory.ru/assets/courseware/v1/36bfee60556351f7853516eb7fd46201/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-ml3-7_6.png)

***
## **ГРАДИЕНТНЫЙ БУСТИНГ В SKLEARN**

Как и все ансамбли, градиентный бустинг находится в модуле **ensemble** библиотеки **sklearn**. В качестве входных данных продолжим использовать данные о диабете.

Градиентный бустинг над деревьями для решения задачи регрессии реализован в классе **GradientBoostingRegressor**. Для задачи классификации данный метод реализован в классе **GradientBoostingClassifier**. Параметры у классов схожи, различия только в функции потерь. Поэтому мы можем рассмотреть параметры **GradientBoostingRegressor**, подразумевая, что у классификатора они идентичны.

Основные параметры **GradientBoostingRegressor**:

* **loss** — функция потерь. По умолчанию в регрессии 'squared_loss' - наша любимая MSE, а в классификации 'deviance' - логистическая функция потерь (logloss).
* **learning_rate** — темп обучения. По умолчанию 0.1. 
* **n_estimators** — количество деревьев в бустинге (число ***K*** из бустинга). По умолчанию равно 100.
* **max_depth** — максимальная глубина одного дерева. По умолчанию равна 3 — строятся короткие деревья с большим смещением.
* **min_samples_leaf** — минимальное число объектов в листе. По умолчанию 1.
* **random_state** — число, отвечающее за генерацию случайных чисел.

✍️ Давайте построим модель градиентного бустинга над деревьями решений. Продолжим использовать данные о диабете:

In [11]:
from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor(
	max_depth=3, #максимальная глубина дерева
n_estimators=50, #количество деревьев
random_state=42 #генератор случайных чисел
)
gb.fit(X_train, y_train)
 
gb_pred  = gb.predict(X_test)
print(f'Качество предсказания по MSE для GradientBoost {round(mean_squared_error(y_test, gb_pred),2)}')

Качество предсказания по MSE для GradientBoost 0.91


Как мы видим, при прочих равных градиентный бустинг даёт меньшую метрику по сравнению с адаптивным. 

⬇️ Однако не стоит радоваться, так как в зависимости от качества данных, от различных параметров, например количества базовых моделей, картина может измениться в обратную сторону. Пример таких параметров представлен в задании ниже. 

In [15]:
# Измените количество базовых моделей до 10 и 100.
# В качестве ответа приведите полученные результаты,
# округленные до второго знака после запятой.
# Если второе число после запятой равно нулю, округлите до одного знака.

gb = GradientBoostingRegressor(
	max_depth=3, #максимальная глубина дерева
n_estimators=10, #количество деревьев
random_state=42 #генератор случайных чисел
)
gb.fit(X_train, y_train)
 
gb_pred  = gb.predict(X_test)
print(f'Качество предсказания по MSE для GradientBoost {round(mean_squared_error(y_test, gb_pred),2)}')

Качество предсказания по MSE для GradientBoost 702.75


Полученное качество говорит о том, что с повышением количества базовых моделей наша модель сходится к решению быстрее прочих моделей. Однако высокая ошибка на маленьком количестве моделей говорит о том, что использования малого количества моделей может не хватить, чтобы модель действительно качественно обучалась, то есть **повышение качества у градиентного бустинга происходит нелинейно.**
***

✍️ Попробуем решить задачу бинарной классификации с помощью градиентного бустинга. Для этого возьмём датасет качества вина. В качестве целевой метки будем считать качественным вино, рейтинг которого больше пяти.

In [17]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

df =pd.read_csv('data/winequality-red.csv', sep = ';')
df.head(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [19]:
# Как и договаривались ранее, хорошим вином будем считать все, имеющие значение признака quality больше 5:
X = df.drop(['quality'],axis = 1)
y = (df['quality'] >5).astype(int)

# Разделим выборку на обучающую и тестовую:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)
 
print(f' Размерность обучающей выборки {X_train.shape}')
print(f' Размерность тестовой выборки {X_test.shape}')

 Размерность обучающей выборки (1071, 11)
 Размерность тестовой выборки (528, 11)


In [22]:
# Инициализируем модель градиентного бустинга:
gb = GradientBoostingClassifier(
    loss='deviance', #функция потерь
    learning_rate=0.1, #темп обучения
    n_estimators=100, #число деревьев
    max_depth=3, #максимальная глубина дерева
    random_state=42 #генератор случайных чисел
)

# Обучим её на тренировочной выборке:
gb.fit(X_train, y_train)

# Посмотрим на метрики классификации с помощью classification_report():
y_pred = gb.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.77      0.77       238
           1       0.81      0.80      0.81       290

    accuracy                           0.79       528
   macro avg       0.79      0.79      0.79       528
weighted avg       0.79      0.79      0.79       528



Аналогично другим классификаторам в sklearn мы можем получать оценки вероятностей для каждого класса с помощью метода **predict_proba()**. Можете использовать эту возможность, например, для подбора оптимального порога вероятности, как мы делали это в предыдущих модулях.

In [23]:
gb.predict_proba(X_test)

array([[0.7575177 , 0.2424823 ],
       [0.81931477, 0.18068523],
       [0.80325657, 0.19674343],
       ...,
       [0.0212709 , 0.9787291 ],
       [0.90355559, 0.09644441],
       [0.80946194, 0.19053806]])

Известно, что с уменьшением темпа обучения (**learning_rate**) возникает необходимость повышения количества базовых моделей, то есть существует некоторая дилемма (trade-off) между темпом обучения и количеством моделей (**n_estimators**). Посмотрим на практике, как эти параметры связаны между собой. 

***
### **ЗАДАЧИ**

Обучите на тренировочной выборке (X_train, y_train) модель градиентного бустинга, состоящего из 200 деревьев, максимальная глубина каждого из которых равна 3. Минимальное число объектов в листе — 10. Темп обучения возьмите равный 0.01. Параметр random_state установите в значение 42.

Сделайте предсказание качества вина для тренировочного и тестового набора данных. Рассчитайте метрику accuracy для тестового набора (X_train, y_train) и запишите её в качестве ответа с точностью до двух знаков после запятой.

In [24]:
# Инициализируем модель градиентного бустинга:
gb = GradientBoostingClassifier(
    loss='deviance', #функция потерь
    learning_rate=0.01, #темп обучения
    n_estimators=200, #число деревьев
    min_samples_leaf=10,
    max_depth=3, #максимальная глубина дерева
    random_state=42 #генератор случайных чисел
)

# Обучим её на тренировочной выборке:
gb.fit(X_train, y_train)

# Посмотрим на метрики классификации с помощью classification_report():
y_pred = gb.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.69      0.74      0.71       238
           1       0.77      0.73      0.75       290

    accuracy                           0.73       528
   macro avg       0.73      0.73      0.73       528
weighted avg       0.73      0.73      0.73       528



Увеличьте количество деревьев до 500, остальные параметры оставьте прежними.

Сделайте предсказание качества вина для тренировочного и тестового набора данных. Рассчитайте метрику accuracy для тестового набора (X_train, y_train) и запишите её в качестве ответа с точностью до двух знаков после запятой.

In [25]:
# Инициализируем модель градиентного бустинга:
gb = GradientBoostingClassifier(
    loss='deviance', #функция потерь
    learning_rate=0.01, #темп обучения
    n_estimators=500, #число деревьев
    min_samples_leaf=10,
    max_depth=3, #максимальная глубина дерева
    random_state=42 #генератор случайных чисел
)

# Обучим её на тренировочной выборке:
gb.fit(X_train, y_train)

# Посмотрим на метрики классификации с помощью classification_report():
y_pred = gb.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.72      0.75      0.73       238
           1       0.79      0.76      0.77       290

    accuracy                           0.75       528
   macro avg       0.75      0.75      0.75       528
weighted avg       0.76      0.75      0.75       528



Увеличьте learning rate до 0.2.

Сделайте предсказание качества вина для тренировочного и тестового набора данных. Рассчитайте метрику accuracy для тестового набора (X_train, y_train) и запишите её в качестве ответа с точностью до двух знаков после запятой.

In [26]:
# Инициализируем модель градиентного бустинга:
gb = GradientBoostingClassifier(
    loss='deviance', #функция потерь
    learning_rate=0.2, #темп обучения
    n_estimators=500, #число деревьев
    min_samples_leaf=10,
    max_depth=3, #максимальная глубина дерева
    random_state=42 #генератор случайных чисел
)

# Обучим её на тренировочной выборке:
gb.fit(X_train, y_train)

# Посмотрим на метрики классификации с помощью classification_report():
y_pred = gb.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.76      0.77       238
           1       0.81      0.83      0.82       290

    accuracy                           0.80       528
   macro avg       0.79      0.79      0.79       528
weighted avg       0.80      0.80      0.80       528



*** 
## **ИТОГИ**

⭐️ Поздравляем! Мы разобрали практически все ныне существующие методы ансамблирования — от самых простых, которые легли в основу случайного леса, до самых передовых (градиентных бустингов). Мы посмотрели на реализации в библиотеке sklearn, постарались разобрать методологию и основные параметры каждой модели. 

Основными рекомендациями по использованию ансамблей является использование случайных лесов и градиентных бустингов в первую очередь для классических задач классификации и регрессии с простыми табличными данными, так как подавляющее большинство соревнований в этой области было выиграно именно благодаря этим двум моделям. 

После данного модуля может сложиться впечатление, что ансамблирование — всегда лучший вариант для финальной модели, однако часто в задачах требуется **интерпретируемость** результатов, которой не могу похвастаться бустинг и стекинг. 

*Кроме того, использование случайного леса на данных с большим количеством выбросов может привести к тому, что модель будет обучается на данных, у которых дисперсия в разы больше, чем у исходных, так как подвыборка для обучения выбирается случайно. Таким образом, мы не сможем получить выигрыш в дисперсии, который нам даёт ансамблирование.*

В заключение приведём таблицу преимуществ и недостатков методов ансамблирования:

![](data/f13.PNG)